# Perform Local Outlier Factor
Gather the created FPD streams and perform the Local Outlier Factor method to detect outliers.

In [1]:
import pandas as pd
import numpy as np
import json
import math
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
import datetime as dt
import time

## Load FPDs
Load the FPDs json file to use in the program

In [2]:
def load_fpds(file):
    with open(file) as f:
        return json.load(f)

## Vectorize FPDs
Vectorize each FPD with the length of the vector as the maximum flow measurement. The goal is to create a range from zero to max and attach the probabilities accordingly. If the value is not in the FPD, the probability is zero.

In [5]:
def vectorize_fpd(fpd):
    vector_list = list(range(max(fpd[0])+1))
    vector = []
    for value in vector_list:
        if value in(fpd[0]):
            vector.append(fpd[1][fpd[0].index(value)])
        else:
            vector.append(0)
    return vector

## Vector sizing
If two vectors are of a different size, then make the shortest vector longer so that both are the same size. Do this by adding zero values to the missing values.

In [6]:
def vector_sizing(vector1,vector2):
    size1,size2 = len(vector1),len(vector2)
    if size1>size2:
        added_values = list(range(size2,size1))
        vector2.extend([0 for added_value in added_values])
        return vector1,vector2
    if size2>size1:
        added_values = list(range(size1,size2))
        vector1.extend([0 for added_value in added_values])
        return vector1,vector2
    else:
        return vector1,vector2

## Bhattacharyya distance for vectors
Function to compute the distance between two fpd vectors. The measure used for this is the Bhattacharyya distance.

In [7]:
def bhatta(vector1,vector2):
    if vector1 == vector2: #redundant as of new sim measure?
        return 0.0
    else:
        try: #This should no longer be needed when we clean the 0 values
            vector1,vector2 = vector_sizing(vector1,vector2)
            v1 = np.average(vector1)
            v2 = np.average(vector2)
            distance = 0
            for i in range(len(vector1)):
                distance += math.sqrt( vector1[i] * vector2[i] )
            distance = math.sqrt( 1 - ( 1 / math.sqrt(v1*v2*len(vector1)*len(vector2)) ) * distance)
            return distance
        except:
            return 0.0

## Similarity measure
Function that uses the previous functions and combines them to be able to calculate the similarity of 1 fpd to all other fpds. Do an all to all calculation of the distances without doing redundant calculations. Use the bhatta function in this function to calculate distances.

In [9]:
def similarity_measure(stream):
    distances_matrix = []
    to_skip = []
    for fpd_one in stream: #for each fpd in stream
        distances_vector = [] #create a vector to be filled with the distances
        fpd_vector1 = vectorize_fpd(stream[fpd_one])
        for fpd_two in stream:
            fpd_vector2 = vectorize_fpd(stream[fpd_two]) #vectorize the item from which to be measured
            to_skip.append([fpd_one,fpd_two])
            distances_vector.append([bhatta(fpd_vector1,fpd_vector2)]) #measure distance and append
        distances_matrix.append(distances_vector)
    return [[value for [value] in vector]for vector in distances_matrix]

# Local Outlier Factor

First get the k closest fpds

Second calculate LRD

Third calculate LOF

In [13]:
def get_KNN(distances_stream,k):
    #Get the k smallest distances in one vector by sorting and then slicing
    #Copy to be sure there are no errors due to sorting
    copy_stream = distances_stream.copy()
    #Add index so that we are able to find the original index before sorting
    indexes = list(range(len(copy_stream)))
    indexed_copy_stream = list(zip(indexes,copy_stream))
    indexed_copy_stream = sorted(indexed_copy_stream, key = lambda x: x[1]) # sort based on second value
    return indexed_copy_stream[1:k+1] #Skip the first one, since that is always self which is 0

In [14]:
def local_reachability_density(f,distances_matrix,k):
    #Sum up all the distances in knn and divide it by k, which is the length of knn and divide by 1
    knn = get_KNN(distances_matrix[f],k) #KNN from the vector of the matrix of f
    knn_distances = [a[1] for a in knn]
    if sum(knn_distances) == 0:
        return 0
    return 1/(sum(knn_distances)/len(knn_distances))

In [15]:
def local_outlier_factor(f,distances_matrix,k):
    #Calculate the lrd of f and calculate the lrd of the knn's then devide these and sum it up.
    lof = 0
    knn = get_KNN(distances_matrix[f],k)
    neighbours = [a[0] for a in knn] #list of indexes of the neighbours
    lrd_one = local_reachability_density(f,distances_matrix,k)
    if lrd_one == 0:
        return 0
    for neighbour in neighbours: #calculate the lrd for each neighbour
        lrd_two = local_reachability_density(neighbour,distances_matrix,k)
        if lrd_two == 0:
            return 0
        fraction = lrd_two/lrd_one
        lof = lof+fraction
    if lof == 0:
        return 0
    return 1/(lof/k)

In [16]:
def perform_lof(stream,k,hour):
    temp = {sum(value[0])/len(value[0]) for (key,value) in stream.items()}
    stream_average = sum(temp)/len(temp)
    dates = list(stream.keys())
    start2 = time.time()
    #Calculate the lof for all fpds in stream and return a list of lof scores with  
    #matching index (could become dates if necessary). 
    lof_scores = []
    #Create distances matrix
    distances = similarity_measure(stream)
    
    for fpd in range(len(distances[0])): #For each fpd
        average = sum(stream[list(stream.keys())[fpd]][0])/len(stream[list(stream.keys())[fpd]][0])
        if average > stream_average:
            high_traffic = 1
        else:
            high_traffic = 0
        lof_scores.append([dates[fpd],hour,local_outlier_factor(fpd,distances,k),high_traffic]) #Calculate lof_scores
    end2 = time.time()
    print('Time taken for this stream: ',end2 - start2)
    return lof_scores #Return list of scores

# Creating outlier lists

In [18]:
def detect_outliers(file,k):
    print('Welcome to the LOF calculation algorithm by Wesley Fitters!')
    start1 = time.time()
    fpd_file = load_fpds(file)
    intersections = list(fpd_file.keys())
    #set up weekdays list containing weekdays that are present in the dict
    weekdays = list(fpd_file[intersections[0]].keys())
    #set up hours list containing hours that are present in the dict
    hours = list(fpd_file[intersections[0]][weekdays[0]].keys())
    #Set up the dictionary to be filled with outliers
    outliers = create_outlier_dict(intersections)
    
    #We feed a single stream to the similarity measure function.
    #Therefore, we should go through the dict and pick up single streams and then feed them.
    for intersection in intersections:
        for weekday in weekdays:
            for hour in hours:
                print('Starting LOF calculation for stream: ',intersection,weekday,hour)
                outliers[intersection][weekday][hour] = list(perform_lof(fpd_file[intersection][weekday][hour],k,hour))
        with open(str(intersection)+'.json', 'w') as f:
            json.dump(outliers, f)
    end1 = time.time()
    print('LOF algorithm is finished! It took ',end1-start1,' to perform the calculations in total.')
    return outliers

In [19]:
def create_outlier_dict(intersections):
    fpds = {}
    hours = [(dt.time(i).strftime('%H')) for i in range(24)] #Create list of hours
    for i in range(len(intersections)): #each intersection
        fpds[intersections[i]] = {}
        for weekday in range(7): #Set up day of the week
            weekday = str(weekday)
            fpds[intersections[i]][weekday] = {}
    return fpds

In [1]:
k=20

In [20]:
outliers = detect_outliers('./edited_data/fpd_collection.json',k)

Welcome to the LOF calculation algorithm by Wesley Fitters!
Starting LOF calculation for stream:  K302 0 00
Time taken for this stream:  5.172491073608398
Starting LOF calculation for stream:  K302 0 01
Time taken for this stream:  4.36456823348999
Starting LOF calculation for stream:  K302 0 02
Time taken for this stream:  3.8495349884033203
Starting LOF calculation for stream:  K302 0 03
Time taken for this stream:  3.695146083831787
Starting LOF calculation for stream:  K302 0 04
Time taken for this stream:  3.124758005142212
Starting LOF calculation for stream:  K302 0 05
Time taken for this stream:  3.9235100746154785
Starting LOF calculation for stream:  K302 0 06
Time taken for this stream:  12.890807867050171
Starting LOF calculation for stream:  K302 0 07
Time taken for this stream:  17.391958951950073
Starting LOF calculation for stream:  K302 0 08
Time taken for this stream:  19.44774103164673
Starting LOF calculation for stream:  K302 0 09
Time taken for this stream:  18.93

KeyboardInterrupt: 

In [ ]:
with open('./edited_data/outliers_collection.json', 'w') as f:
    json.dump(outliers, f)